In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import math
import torch
import torch.nn.functional as F
import llama_cpp

from path import *

In [3]:
phrase = "Italy"
k = 3
max_depth = 6
G = phrase_graph(phrase, k, max_depth)

In [4]:
pos = nx.multipartite_layout(G, subset_key="depth", scale=3)
for node, position in pos.items():
    G.nodes[node]['pos'] = position

ggml_backend_cuda_buffer_type_alloc_buffer: allocating 192.00 MiB on device 0: cudaMalloc failed: out of memory
llama_kv_cache_init: failed to allocate buffer for kv cache
llama_new_context_with_model: llama_kv_cache_init() failed for self-attention cache
[autoreload of path failed: Traceback (most recent call last):
  File "/home/marco/miniconda3/envs/epfl/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/marco/miniconda3/envs/epfl/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/marco/miniconda3/envs/epfl/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 995, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call

In [15]:
import networkx as nx
import plotly.graph_objects as go
from ipywidgets import widgets

seed_node = (phrase, 0)
# Assume G is your graph and phrase is defined

# Create a dictionary to store all paths from each depth 5 node to the seed
all_paths_to_seed = {node: list(nx.all_simple_paths(G, seed_node, node)) for node in G.nodes() if G.nodes[node]['depth'] == max_depth}

# Create edge traces for the graph structure
edge_x, edge_y = [], []
for edge in G.edges():
    x0, y0 = G.nodes[edge[0]]['pos']
    x1, y1 = G.nodes[edge[1]]['pos']
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])

edge_trace = go.Scatter(x=edge_x, y=edge_y, line=dict(width=0.5, color='#888'),
                        hoverinfo='none', mode='lines')

# Create node trace
node_x, node_y, node_text = [], [], []
for node in G.nodes():
    x, y = G.nodes[node]['pos']
    node_x.append(x)
    node_y.append(y)
    node_text.append(f"{node[0]} (Depth: {G.nodes[node]['depth']})")

node_trace = go.Scatter(
    x=node_x, y=node_y,
    mode='markers',
    hoverinfo='text',
    text=node_text,
    marker=dict(
        showscale=True,
        colorscale='YlGnBu',
        reversescale=True,
        color=[G.nodes[node]['depth'] for node in G.nodes()],
        size=10,
        colorbar=dict(thickness=15, title='Node Depth', xanchor='left', titleside='right'),
        line_width=2
    )
)

# Create figure
fig = go.FigureWidget(data=[edge_trace, node_trace],
                layout=go.Layout(
                    showlegend=False,
                    hovermode='closest',
                    margin=dict(b=20,l=5,r=5,t=40),
                    xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                    yaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                ))

# Create a widget to display clicked node info
click_info = widgets.Label(value="Click a depth 5 node to highlight paths")

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

IndexError: tuple index out of range

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

IndexError: tuple index out of range

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

...

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

IndexError: tuple index out of range

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

IndexError: tuple index out of range

Click event: trace=Scatter({
    'hoverinfo': 'text',
    'marker': {'color': [0, 1, 1, ..., 6, 6, 6],
               'colorbar': {'thickness': 15, 'title': {'side': 'right', 'text': 'Node Depth'}, 'xanchor': 'left'},
               'colorscale': [[0.0, 'rgb(255,255,217)'], [0.125,
                              'rgb(237,248,177)'], [0.25, 'rgb(199,233,180)'],
                              [0.375, 'rgb(127,205,187)'], [0.5,
                              'rgb(65,182,196)'], [0.625, 'rgb(29,145,192)'],
                              [0.75, 'rgb(34,94,168)'], [0.875, 'rgb(37,52,148)'],
                              [1.0, 'rgb(8,29,88)']],
               'line': {'width': 2},
               'reversescale': True,
               'showscale': True,
               'size': 10},
    'mode': 'markers',
    'text': [Italy (Depth: 0), , (Depth: 1),  and (Depth: 1), ...,  an (Depth: 6),
             us (Depth: 6),  me (Depth: 6)],
    'uid': 'c99a2776-5167-4d18-a716-bfb6772e89bd',
    'x': [-0.1314471

In [16]:

def custom_click_handler(trace, points, selector):
    
    print(f"Click event: trace={trace}\n\npoints={points}\n\nselector={selector}")
    
    # if not points.point_inds:
    #     click_info.value = "No point selected"
    #     return
    
    try:
        clicked_index = points.point_inds[0]
        all_nodes = list(G.nodes())
        if clicked_index >= len(all_nodes):
            click_info.value = "Invalid node index"
            return
        
        clicked_node = all_nodes[clicked_index]
        node_depth = G.nodes[clicked_node]['depth']
        
        click_info.value = f"Clicked node: {clicked_node[0]} (Depth: {node_depth})"    
        
        with fig.batch_update():
            # Always remove previously highlighted paths
            fig.data = fig.data[:2]  # Keep only edge_trace and node_trace
            
            if node_depth == max_depth:
                paths = all_paths_to_seed.get(clicked_node, [])
                num_paths = len(paths)
                click_info.value += f" - Found {num_paths} path{'s' if num_paths != 1 else ''}"
                
                # Highlight new paths
                for i, path in enumerate(paths):
                    x = [G.nodes[n]['pos'][0] for n in path]
                    y = [G.nodes[n]['pos'][1] for n in path]
                    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', line=dict(color='red', width=2),
                                            hoverinfo='none', name=f'Path {i+1} to {clicked_node[0]}'))
            else:
                click_info.value += f" - Not a depth {max_depth} node, no paths to highlight"

    except Exception as e:
        click_info.value = f"An unexpected error occurred: {type(e).__name__}: {str(e)}"


# Attach the custom click event handler
fig.data[1].on_click(custom_click_handler)

display(widgets.VBox([fig, click_info]))

    'data': [{'hoverinfo': 'none',
              'line': {'color': '#888', 'widt…

In [ ]:
# import cProfile
# import pstats
# from pstats import SortKey

# cProfile.run("G = phrase_graph(phrase, k, max_depth)", "stats")
# p = pstats.Stats("stats")
# p.sort_stats(SortKey.TIME)
# p.print_stats()